In [ ]:
# Data Preprocessing

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import nltk #language processing
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

import nltk
import pandas as pd
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize

import re
import emoji


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [ ]:
data_train = pd.read_csv("congressional_tweet_training_data.csv",names=None)
data_test= pd.read_csv("congressional_tweet_test_data.csv",names=None)

In [ ]:
data_train.head()

In [ ]:
data_train['party_class']=data_train['party_id'].apply(lambda x: 0 if x=='D' else 1)

In [ ]:
data_test['party_class']=data_test['party'].apply(lambda x: 0 if x=='D' else 1)

In [ ]:
# Tokenize
tokens_list=[word_tokenize(sent) for sent in data_train['full_text']]

In [ ]:
# Pos Tag 
tagged_tokens =[pos_tag(sent) for sent in tokens_list]

In [ ]:
def get_wordnet_pos(treebank_tag): #Following from [16]

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None # for easy if-statement 

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
x=pd.DataFrame(data_train['full_text'])

In [ ]:
x['tagged_tokens']=tagged_tokens

In [ ]:
lemma_list = x['tagged_tokens'].transform(lambda value: ' '.join([lemmatizer.lemmatize(a[0],pos=get_wordnet_pos(a[1])) if get_wordnet_pos(a[1]) else a[0] for a in  value]))

In [ ]:
x['lemma_list']=lemma_list

In [ ]:
x.head()

In [ ]:
# Remove numbers, words <2 characters, punctuation, links and emojis 

def emoji_free_text(text): # From [9] 
    return emoji.get_emoji_regexp().sub(r'', text)

In [ ]:
def clean_text(text: str) -> str:
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    cleaned_tokens = []
    for token in tokens:
        lower_token = token.lower()
        len_check = len(lower_token) >= 2
        #start_check = not (lower_token.startswith('http') or lower_token.startswith('\\') or lower_token.startswith(':'))
        stop_word_check = lower_token not in stop_words
        if len_check and stop_word_check:
            lower_token=re.sub(r'\d+', '', lower_token) 
            lower_token=re.sub(r'\b\w{1}\b', '',  lower_token)  
            lower_token=re.sub(r'[^\w\s]', '', lower_token)                 # Remove punctuation
            lower_token=re.sub(r'http\S+', '',lower_token)                 # Remove links
            cleaned_tokens.append(re.sub('[,.!?]|<br \/>\+|<br \/>', '', lower_token))
    return ' '.join(cleaned_tokens)


In [ ]:
Text_clean = x['lemma_list'].map(clean_text)
Text_cleaner = Text_clean .map(emoji_free_text)
data_train['text_clean']=Text_cleaner

In [ ]:
data_train.head()

In [ ]:
# Cleaning test data set

# test datasindaki naleri de silecek miyiz?

# Tokenize
tokens_list_test=[word_tokenize(sent) for sent in data_test['full_text']]

In [ ]:
# Pos Tag 
tagged_tokens_test =[pos_tag(sent) for sent in tokens_list_test]

In [ ]:
x2=pd.DataFrame(data_test['full_text'])

In [ ]:
x2['tagged_tokens_test']=tagged_tokens_test

In [ ]:
lemma_list_test = x2['tagged_tokens_test'].transform(lambda value: ' '.join([lemmatizer.lemmatize(a[0],pos=get_wordnet_pos(a[1])) if get_wordnet_pos(a[1]) else a[0] for a in  value]))

In [ ]:
x2['lemma_list_test']=lemma_list_test

In [ ]:
Text_clean_test = x2['lemma_list_test'].map(clean_text)
Text_cleaner_test = Text_clean_test .map(emoji_free_text)
data_test['text_clean_test']=Text_cleaner_test

In [ ]:
data_test

#### Naive Bayes with Clean Data

In [ ]:
X_train = data_train['text_clean']
y_train = data_train['party_class']

X_test = data_test['text_clean_test']

In [ ]:
text=data_train['text_clean'] 
text2=data_test['text_clean_test'] 
frames = [text, text2]
  
result = pd.concat(frames)
display(result)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer()
X=tfidf.fit_transform(result)

In [ ]:
X_train=X[0:592803]
y_train=data_train['party_class']
X_test=X[592803:857803]

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
clf=MultinomialNB()

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
y_test=clf.predict(X_test)

In [ ]:
y_test=pd.DataFrame(data=y_test, columns=['party'])

In [ ]:
y_test['party']=y_test3['party'].apply(lambda x: 'D' if x==0 else 'R')

In [ ]:
y_test

In [ ]:
y_test.to_csv('ytest_nb_clean_text_istanbul.csv')